In [1]:
# Achtung normalize und invert sind etwas anders weil pos schon auf 0,1
# Alle Trajektorien sind in einem file für die 6 input arten
# 900 Traj mit je 500 zeitschritten
# die Testtrajektorien sind immer die Enden der 30 Trajektorien (6*5) und 7mal länger

# Vorher T  = [0 , 0.0112] 500 steps
# Jetzt T = [0, 0.0224] 500 steps 
# => Schrittweite verdoppelt    
# Modell normal trainieren und dann bei Vergleich zu Simulink die Simulink Daten interpolieren damit sie gleich lang sind!

In [6]:
from meas_get_data import *
from meas_test_func_fs import * 
from meas_get_data import *
import os
torch.set_default_dtype(torch.float64)
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
path = r"C:\Users\strasserp\Documents\ventil_lstm\Experiment_Meassurements\Messungen\messdaten_900traj_500steps.csv"
data =get_data(path, num_inits=100)
data.size()

path = r"C:\Users\strasserp\Documents\ventil_lstm\Experiment_Meassurements\Messungen\Test-Messdaten_30traj_7times500steps.csv"
test_data =get_data(path, num_inits=100)
test_data.size()

In [ ]:
visualise(normalize_invert(test_data),3)
visualise(normalize_invert(test_data),4)
visualise(normalize_invert(test_data),5)

In [ ]:
visualise(normalize_invert(data),89)
visualise(normalize_invert(data),91)
visualise(normalize_invert(data),92)

In [ ]:

params_lstm =    {
                        "window_size" : 16,
                        "h_size" : 8,
                        "l_num" : 3,
                        "learning_rate" : 0.001,
                        "batch_size" : 20,

                        "percentage_of_data" : 0.8,
                        "cut_off_timesteps" : 0,
                        "part_of_data" : 0,
                        "epochs" : 1000,
                        "test_every_epochs" : 100,

                        "T_max" : 500,

                        "experiment_number" : np.random.randint(0,1000)
                        }


if os.name == "nt":
    path_train_data=r"C:\Users\strasserp\Documents\ventil_lstm\Experiment_Meassurements\Messungen\messdaten_900traj_500steps.csv"
    path_test_data = r"C:\Users\strasserp\Documents\ventil_lstm\Experiment_Meassurements\Messungen\TEST-Messdaten_30traj_7times500steps.csv"

else:
    path_train_data=r"/home/rdpusr/Documents/ventil_lstm/Experiment_Meassurements/Messungen/messdaten_900traj_500steps.csv"
    path_test_data = r"/home/rdpusr/Documents/ventil_lstm/Experiment_Meassurements/Messungen/TEST-Messdaten_30traj_7times500steps.csv"

test_data = get_data(path_test_data,num_inits=100)
train_data = get_data(path_train_data,num_inits=100)

 

model_lstm = OR_LSTM(input_size=4, hidden_size=params_lstm["h_size"], out_size=2, layers=params_lstm["l_num"], window_size=params_lstm["window_size"]).to(device)

path_lstm=r"C:\Users\strasserp\Documents\ventil_lstm\Experiment_Meassurements\or_lstm_0_3.pth"
model_lstm.load_state_dict(torch.load(path_lstm, map_location=torch.device(device)))

#test(test_data.to(device), model_lstm, model_type="lstm", window_size=params_lstm["window_size"], display_plots=True, numb_of_inits = 1, fix_random=True)
test(test_data.to(device), model_lstm, model_type="lstm", window_size=params_lstm["window_size"], display_plots=True, numb_of_inits = 5, fix_random=True)